In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs
import keras
from datetime import datetime
from sklearn import preprocessing

In [2]:
# Baca data
data = pd.read_csv(r'C:\Users\Gus Ary\Downloads\dataset_kuesioner.xlsx - Sheet1.csv', sep=",", encoding='utf8', engine="python")

# Menghapus data yang kosong
data.dropna(inplace=True)

# Preprocessing kategori
data.rename(columns={'category 1': 'category_1', 'category 2': 'category_2'}, inplace=True)
le = preprocessing.LabelEncoder()
data['category_1'] = le.fit_transform(data['category_1'])
data['category_2'] = le.fit_transform(data['category_2'])

In [3]:
# Menambahkan kolom "nomor" di sebelah kiri kolom "judul"
data.insert(0, 'nomor', range(1, len(data) + 1))

# Baca data kedua
data2 = pd.read_csv(r'C:\Users\Gus Ary\Downloads\dataset_test(100user).csv')

# Install library tensorflow-recommenders jika belum terinstall
# pip install tensorflow-recommenders

data2.rename(columns={'user id': 'user_id'}, inplace=True)

In [4]:
data2

,user_id,kategori,history
0,1,Kesehatan,7.0
1,1,Pendidikan,4.0
2,1,Hukum,8.0
3,1,Keuangan,5.0
4,1,Pariwisata,7.0
...,...,...,...
1495,100,Agama dan Kepercayaan,7.0
1496,100,Bisnis dan Industri,10.0
1497,100,Politik dan Pemerintahan,9.0
1498,100,Transportasi dan Logistik,2.0


In [5]:
# Data preprocessing
data2['user_id'] = data2['user_id'].astype(str)
data2['kategori'] = data2['kategori'].astype(str)
data2['history'] = data2['history'].astype(np.float32)

# Membuat dataset
dataset = tf.data.Dataset.from_tensor_slices((
    tf.cast(data2['user_id'].values.reshape(-1, 1), tf.string),
    tf.cast(data2['kategori'].values.reshape(-1, 1), tf.string),
    tf.cast(data2['history'].values.reshape(-1, 1), tf.float32)
))

In [6]:
@tf.function
def rename(x0, x1, x2):
    y = {}
    y["user_id"] = x0
    y['kategori'] = x1
    y['history'] = x2
    return y


dataset = dataset.map(rename)

# Mengambil unique kategori dan user_id
kategori = data2.kategori.values
userid = data2.user_id.values

unique_kategori = np.unique(list(kategori))
unique_userid = np.unique(list(userid))

In [7]:

class RankingModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        embedding_dimension = 10

        # Compute embeddings for users.
        self.userid_embeddings = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.StringLookup(
                vocabulary=unique_userid, mask_token=None),
            tf.keras.layers.Embedding(len(unique_userid) + 1, embedding_dimension)
        ])

        # Compute embeddings for kategori.
        self.kategori_embeddings = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.StringLookup(
                vocabulary=unique_kategori, mask_token=None),
            tf.keras.layers.Embedding(len(unique_kategori) + 1, embedding_dimension)
        ])

        # Compute predictions.
        self.ratings = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation="relu"),
            tf.keras.layers.Dense(32, activation="relu"),
            tf.keras.layers.Dense(1)
        ])
        
    
    def call(self, inputs):
        user_id, kategori = inputs
        userid_embedding = self.userid_embeddings(user_id)
        kategori_embedding = self.kategori_embeddings(kategori)

        return self.ratings(tf.concat([userid_embedding, kategori_embedding], axis=1))

In [8]:
class userModel(tfrs.models.Model):
    def __init__(self):
        super().__init__()
        self.ranking_model: tf.keras.Model = RankingModel()
        self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )

    def compute_loss(self, features, training=False) -> tf.Tensor:
        user_id_embedded = self.ranking_model.userid_embeddings(features["user_id"])
        kategori_embedded = self.ranking_model.kategori_embeddings(features["kategori"])

        mission_predictions = self.ranking_model.ratings(tf.concat([user_id_embedded, kategori_embedded], axis=1))

        # Compute the loss and metrics
        return self.task(labels=features["history"], predictions=mission_predictions)


In [9]:

model = userModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [10]:
# Cache the dataset
cache_dataset = dataset.cache()

In [11]:
# Tensorboard
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Training
model.fit(cache_dataset, epochs=300, verbose=1, callbacks=[tensorboard_callback])

# Evaluate the model
eval_dataset = dataset.shuffle(len(data2)).batch(1)
eval_results = model.evaluate(eval_dataset, return_dict=True)
print('Evaluation results:', eval_results)

Epoch 1/300
1500/1500 [==============================] - 5s 2ms/step - root_mean_squared_error: 3.0429 - loss: 9.2535 - regularization_loss: 0.0000e+00 - total_loss: 9.2535
Epoch 2/300
1500/1500 [==============================] - 5s 3ms/step - root_mean_squared_error: 2.9396 - loss: 8.6358 - regularization_loss: 0.0000e+00 - total_loss: 8.6358
Epoch 3/300
1500/1500 [==============================] - 4s 3ms/step - root_mean_squared_error: 2.8970 - loss: 8.3868 - regularization_loss: 0.0000e+00 - total_loss: 8.3868
Epoch 4/300
1500/1500 [==============================] - 3s 2ms/step - root_mean_squared_error: 2.8230 - loss: 7.9638 - regularization_loss: 0.0000e+00 - total_loss: 7.9638
Epoch 5/300
1500/1500 [==============================] - 3s 2ms/step - root_mean_squared_error: 2.7407 - loss: 7.5062 - regularization_loss: 0.0000e+00 - total_loss: 7.5062
Epoch 6/300
1500/1500 [==============================] - 4s 2ms/step - root_mean_squared_error: 2.6575 - loss: 7.0577 - regularization_

In [14]:
# Create array with users id in every place on the length of the unique kategori number
user8 = np.array([["8"] for i in range(len(unique_kategori))])

# Convert it to tf.data.Dataset
test_data = tf.data.Dataset.from_tensor_slices((tf.cast(user8.reshape(-1, 1), tf.string),
                                                tf.cast(unique_kategori.reshape(-1, 1), tf.string)))

# Name the columns
@tf.function
def rename_test(x0, x1):
    y = {}
    y["user_id"] = x0
    y['kategori'] = x1
    return y


test_data = test_data.map(rename_test)

# Now let's make predictions and store them in a dictionary
test_mission = {}
for b in test_data:
    user_id_embedded = model.ranking_model.userid_embeddings(b['user_id'])
    kategori_embedded = model.ranking_model.kategori_embeddings(b['kategori'])
    prediction = model.ranking_model.ratings(tf.concat([user_id_embedded, kategori_embedded], axis=1))

    test_mission[b['kategori'].numpy()[0]] = prediction.numpy()[0][0]

# Sort the predictions by score and print the titles
for b in sorted(test_mission, key=test_mission.get, reverse=True):
    print(b)

b'Bisnis dan Industri'
b'Pertanian dan Perikanan'
b'Hukum'
b'Politik dan Pemerintahan'
b'Transportasi dan Logistik'
b'Keuangan'
b'Kesehatan'
b'Teknologi Informasi dan Komunikasi'
b'Agama dan Kepercayaan'
b'Olahraga dan Rekreasi'
b'Pendidikan'
b'Lingkungan dan Konservasi'
b'Pariwisata'
b'Sosial dan Kemanusiaan'
b'Seni dan Budaya'
